In [ ]:
text_len = pd.DataFrame(columns = ['text_length_words', 'text_length_chars'])

text_len['text_length_words'] = df['text'].apply(lambda x: len(x.split()))
text_len['text_length_chars'] = df['text'].apply(len)
text_len

In [ ]:
import pandas as pd 

df = pd.read_csv('IMDB_movie_details.csv')
df.head()

In [ ]:
set([genre for genres in df.genre for genre in genres])

In [ ]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.tokenize import word_tokenize

nltk.download('stopwords')
stop_words = set(stopwords.words('english'))


def preprocessing(corpus): #revise this
    stop_words = set(stopwords.words('english'))
    lemmatizer = WordNetLemmatizer()
    corpus = re.sub(r'[^\w\s]', '', corpus)
    tokens = word_tokenize(corpus)
    tokens = [word.lower() for word in tokens]
    tokens = [word.replace(',' , '') for word in tokens if word not in stop_words]
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    return tokens

df['processed'] = df.plot_summary.astype(str).apply(lambda x: preprocessing(x))

In [ ]:
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

df['embeddings'] = model.encode([' '.join(sentence) for sentence in df['processed'].values])



In [ ]:
import networkx as nx

G = nx.Graph()


[G.add_node(index, title = row['title'], genre = row['genre'], plot_summary = row['plot_summary']) for index, row in df.iterrows()]

for index, row in df.iterrows():
    for i1, r1 in df[index+1:].iterrows():
        cosim =  util.pytorch_cos_sim(row['embeddings'] , r1['embeddings'])
        if cosim > 0.44:
            G.add_edge(index, i1, weight = cosim[0][0])

#remove nodes with no edges
G.remove_nodes_from(list(nx.isolates(G)))



In [ ]:
G.remove_node(832)

In [ ]:
import matplotlib.pyplot as plt
degrees = [G.degree(n) * 100 for n in G.nodes()]  # Scale by 10 for better visualization

# Draw the graph
plt.figure(figsize=(20, 20))
nx.draw(G, node_size=degrees, with_labels=True, font_weight='bold')

In [ ]:
A_d = nx.adjacency_matrix(G)
print(A_d.todense())

In [ ]:
import seaborn as sns

A_d = nx.adjacency_matrix(G)
sns.heatmap(A_d.todense(),
            annot=True,
            cmap = 'viridis',
            xticklabels=G.nodes(),
            yticklabels=G.nodes())

In [ ]:
betw_centr = dict(sorted(nx.betweenness_centrality(G).items(), key=lambda x: x[1], reverse = True))

print(betw_centr)


In [ ]:
close_cent = dict(sorted(nx.closeness_centrality(G).items(), key=lambda x: x[1], reverse = True))
close_cent